# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

# Low Glucose Scenario

## Connect to BioExplorer backend

In [ ]:
from bioexplorer import BioExplorer, RNASequence, Protein, AssemblyProtein, Virus, Surfactant, Membrane, Cell, Sugars, \
    Volume, MeshBasedMembrane, Vector2, Vector3, Quaternion
import nglview

hostname = 'localhost:5000'
resource_folder = '../../tests/test_files/'
    
be = BioExplorer(hostname)
print(be.version())
be.reset()

export_to_cache = False

## Global parameters

In [ ]:
# Scene
scene_size = 800.0

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS

# Glycans
add_glycans = True

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False

# Immune system
nb_glucoses = 120000
nb_lactoferrins = 150
nb_defensins = 300
nb_defensins_on_virus = 2

## Resources

In [ ]:
cache_folder = resource_folder + 'caches/0.9.0/'
image_folder = resource_folder + 'images/'
pdb_folder = resource_folder + 'pdb/'
rna_folder = resource_folder + 'rna/'
obj_folder = resource_folder + 'obj/'
glycan_folder = pdb_folder + 'glycans/'

complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

lymphocyte_path = obj_folder + 'lymphocyte.obj'

## Coronaviruses

In [ ]:
be.add_coronavirus(
    name='Coronavirus 1', resource_folder=resource_folder,
    position=Vector3(-5.0, 19.0, -36.0), rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)
be.add_coronavirus(
    name='Coronavirus 2', resource_folder=resource_folder,
    position=Vector3(73.0, 93.0, -115.0), rotation=Quaternion(0.0, 0.0, 1.0, 0.0),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)
be.add_coronavirus(
    name='Coronavirus 3', resource_folder=resource_folder,
    position=Vector3(-84.0, 110.0, 75.0), rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)
be.add_coronavirus(
    name='Coronavirus 4', resource_folder=resource_folder,
    position=Vector3(-70.0, -100.0, 230.0),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)
be.add_coronavirus(
    name='Coronavirus 5', resource_folder=resource_folder,
    position=Vector3(200.0, 20.0, -150.0), rotation=Quaternion(0.0, 0.707, 0.707, 0.0),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)

In [26]:
be.set_models_visibility(True)

{'contents': 'OK', 'status': True}

In [ ]:
nglview.show_file(pdb_folder + 'sars-cov-2-v1.pdb')

In [ ]:
nglview.show_file(pdb_folder + '6vyb.pdb')

In [ ]:
nglview.show_file(pdb_folder + 'QHD43419a.pdb')

In [ ]:
nglview.show_file(pdb_folder + 'QHD43418a.pdb')

In [ ]:
if export_to_cache:
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    cache_filename = cache_folder + 'low_glucose_viruses.bioexplorer'
    if add_rna:
        cache_filename = cache_folder + 'low_glucose_open_viruses.bioexplorer'
    print(cache_filename)
    be.export_to_cache(cache_filename)

## Host cell

In [ ]:
def add_cell(large_cell, name, size, height, receptors_only, position=Vector3()):
    nb_receptors = 20
    nb_lipids = 300000
    cell_size = size
    if large_cell:
        cell_size *= 2
        nb_lipids *= 4

    if receptors_only:
        nb_lipids = 1
    else:
        nb_receptors = 0
    
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m18.pdb'], occurences=nb_receptors,
        position=Vector3(0.0, 6.0, 0.0))
    
    membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'], occurences=nb_lipids)

    cell = Cell(
        name=name, size=cell_size, extra_parameters=[height],
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL, 
        membrane=membrane, receptor=ace2_receptor)
    
    be.add_cell(
        cell=cell, position=position,
        representation=protein_representation,
        random_seed=1)

    if nb_receptors != 0 and add_glycans:
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name, 
            glycan_type=be.NAME_GLYCAN_COMPLEX,
            protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
            indices=[53, 90, 103, 322, 432, 690])
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name,
            glycan_type=be.NAME_GLYCAN_HYBRID,
            protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
            indices=[546])

        indices = [[155, Quaternion(0.707, 0.0, 0.707, 0.0)],
                   [730, Quaternion(0.707, 0.0, 0.707, 0.0)]]
        for index in indices:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_RECEPTOR, representation=glycan_representation,
                chain_ids=[2, 4], site_indices=[index[0]], rotation=index[1])
            be.add_sugars(o_glycan)

In [ ]:
cell_position = Vector3(4.5, -186.0, 7.0)
cell_height = scene_size / 10.0
add_cell(
    large_cell=False,
    name='Cell_receptors', receptors_only=True,
    size=scene_size, height=cell_height,
    position=cell_position)
add_cell(
    large_cell=True,
    name='Cell_membrane', receptors_only=False,
    size=scene_size, height=cell_height,
    position=cell_position)

In [ ]:
if export_to_cache:
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    be.set_protein_color_scheme(
        'Cell_receptors', 'Cell_receptors' + '_' + be.NAME_RECEPTOR,
        be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)
    cache_filename = cache_folder + 'cell.bioexplorer'
    print(cache_filename)
    be.export_to_cache(cache_filename)

## Immune system

### Surfactant A and D

In [27]:
nglview.show_file(surfactant_head_source)

NGLWidget()

In [28]:
def add_surfactant_d(name, position, random_seed):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, representation=protein_representation, 
        position=position, random_seed=random_seed)

add_surfactant_d(
    name='Surfactant-D 1', position=Vector3(74.0,  24.0, -45.0), random_seed=1)
add_surfactant_d(
    name='Surfactant-D 2', position=Vector3(-30.0,  91.0,  20.0), random_seed=2)
add_surfactant_d(
    name='Surfactant-D 3', position=Vector3(-165.0, 140.0, 105.0), random_seed=1)
add_surfactant_d(
    name='Surfactant-D 4', position=Vector3(-260.0,  50.0, 150.0), random_seed=2)

In [29]:
def add_surfactant_a(name, position, random_seed):
    surfactant_a = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_A, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_a, representation=protein_representation, 
        position=position, random_seed=random_seed)

add_surfactant_a(
    name='Surfactant-A 1', position=Vector3(200.0,  50.0, 150.0), random_seed=2)

### Glucose

In [30]:
glucose = Protein(
    sources=[glucose_path], load_non_polymer_chemicals=True, 
    occurences=nb_glucoses)
volume = Volume(
    name=be.NAME_GLUCOSE, size=scene_size, protein=glucose)
be.add_volume(
    volume=volume, representation=protein_representation,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0))

{'contents': '', 'status': True}

In [31]:
nglview.show_file(glucose_path)

NGLWidget()

### Lactoferrins

In [32]:
lactoferrin = Protein(
    sources=[lactoferrin_path], load_non_polymer_chemicals=True,
    occurences=nb_lactoferrins)
lactoferrins_volume = Volume(
    name=be.NAME_LACTOFERRIN, size=scene_size, protein=lactoferrin)
be.add_volume(
    volume=lactoferrins_volume, representation=protein_representation,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0), random_seed=2)

{'contents': '', 'status': True}

In [33]:
nglview.show_file(lactoferrin_path)

NGLWidget()

### Defensins

In [34]:
defensin = Protein(
    sources=[defensin_path], load_non_polymer_chemicals=True, 
    occurences=nb_defensins)
defensins_volume = Volume(
    name=be.NAME_DEFENSIN, size=scene_size, protein=defensin)
be.add_volume(
    volume=defensins_volume, representation=protein_representation,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0), random_seed=3)

{'contents': '', 'status': True}

In [35]:
nglview.show_file(defensin_path)

NGLWidget()

### Lymphocyte

In [36]:
clip_planes = [
    [1.0, 0.0, 0.0, scene_size + 5],
    [0.0, 0.0, 1.0, scene_size + 5],
    [0.0, 0.0, -1.0, scene_size + 5],
    [-1.0, 0.0, 0.0, scene_size + 5],
    [0.0, 1.0, 0.0, 250.0],
    [0.0, -1.0, 0.0, 550.0]
]
for plane in clip_planes:
    be.core_api().add_clip_plane(plane)

In [37]:
name = 'Emile'
protein_source = pdb_folder + 'membrane/popc.pdb'
# mesh_source = obj_folder + 'lymphocyte.obj'
mesh_source = '/media/favreau/Documents/medias/covid19/obj/lymphocyte.obj'

position = Vector3(-1935.0, 0.0, 0.0)
rotation = Quaternion(0.707, 0.707, 0.0, 0.0)
scale = Vector3(1.0, 1.0, 1.0)

mesh_based_membrane = MeshBasedMembrane(
    mesh_source=mesh_source, protein_source=protein_source, 
    density=5, atom_radius_multiplier=1.0, 
    surface_variable_offset=5.0)

status = be.add_mesh_based_membrane(
    name, mesh_based_membrane, position=position, rotation=rotation, scale=scale)

In [38]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name, 
    color_scheme=be.COLOR_SCHEME_RESIDUES,
    palette_name='rainbow_r', palette_size=5)

### Materials

In [ ]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_PERLIN,
    user_parameter=0.03, specular_exponent=5, glossiness=0.1)

In [ ]:
for model in be.core_api().scene.models:
    model_id = model['id']
    model_name = model['name']
    if be.NAME_COLLAGEN in model_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)
        print(model_name)
        palette = list()
        emissions = list()
        for i in range(nb_materials):
            palette.append([1,1,1])
            emissions.append(0.2)
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette,
            emissions=emissions
        )        
    if "Emile" in model_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)-1
        print(model_name)
        palette = list()
        a = 0.25
        for i in range(nb_materials):
            palette.append([a * 0.78,a * 0.47,a * 0.34])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette,
        )
        
be.core_api().scene.commit()

In [ ]:
if export_to_cache:
    cache_filename = cache_folder + 'low_glucose_immune.bioexplorer'
    print(cache_filename)
    be.export_to_cache(cache_filename)

## Rendering settings

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=1, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.4
params.gi_distance = 100
params.shadows = 1.0
params.soft_shadows = 1.0
params.fog_start = 1300
params.fog_thickness = 1300
params.max_bounces = 1
be.core_api().set_renderer_params(params)

## Save full scene to cache file

In [ ]:
if export_to_cache:
    cache_filename = cache_folder + 'low_glucose_scenario.bioexplorer'
    print(cache_filename)
    be.export_to_cache(cache_filename)